# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [15]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451853120 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/joebooth/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1451853120 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to joe@joebooth.com and will expire on October 09, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-926 - Server binary: /Users/joebooth/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451853120.log
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.475363     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.513679     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [12]:
model_all.coefficients.print_rows(100)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [13]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [48]:
results = np.full(ki,np.finfo(np.float).max)

In [113]:
ki = 13
penalties = np.empty(ki)
results = np.full(ki,np.finfo(np.float).max)
models = list()
i = 0;
for l1_penalty in np.logspace(1, 7, num=ki):
#     * Fit a regression model with a given `l1_penalty` on TRAIN data. 
#       Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    m = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l1_penalty=l1_penalty, l2_penalty=0., 
                                              verbose = False)
    models.append(m)
#     * Compute the RSS on VALIDATION data 
#       (here you will want to use `.predict()`) for that `l1_penalty`
        # Compute validation error using validation set just formed
    predict = m.predict(validation)
    score = predict - validation['price'] 
    validation_rss = (score*score).sum()
        
    results[i] = validation_rss
    penalties[i] = l1_penalty
    if (np.amin(results) == validation_rss):
        min_l1 = l1_penalty
        min_result = validation_rss
        min_i = i
    print (i, l1_penalty, validation_rss)
    i += 1

# * Report which `l1_penalty` produced the lowest RSS on validation data.

# When you call `linear_regression.create()` make sure you set `validation_set = None`.

# Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`    
    

(0, 10.0, 625766285142459.9)
(1, 31.622776601683793, 625766285362394.1)
(2, 100.0, 625766286057885.0)
(3, 316.22776601683796, 625766288257224.6)
(4, 1000.0, 625766295212186.8)
(5, 3162.2776601683795, 625766317206080.5)
(6, 10000.0, 625766386760658.1)
(7, 31622.776601683792, 625766606749278.5)
(8, 100000.0, 625767302791634.1)
(9, 316227.76601683791, 625769507643886.2)
(10, 1000000.0, 625776517727024.0)
(11, 3162277.6601683795, 625799062845467.0)
(12, 10000000.0, 625883719085425.2)


In [114]:
print(results)
print(penalties)
# print(models)

[  6.25766285e+14   6.25766285e+14   6.25766286e+14   6.25766288e+14
   6.25766295e+14   6.25766317e+14   6.25766387e+14   6.25766607e+14
   6.25767303e+14   6.25769508e+14   6.25776518e+14   6.25799063e+14
   6.25883719e+14]
[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [115]:
# What was the best value for the l1_penalty?
print (min_i, min_l1, min_result)

(0, 10.0, 625766285142459.9)


In [116]:
# What is the RSS on TEST data of the model with the best l1_penalty?
predict = models[min_i].predict(testing)
score = predict - testing['price'] 
testing_rss = (score*score).sum()
print (testing_rss)

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [117]:
models[min_i].coefficients.print_rows(100)
# models[12]
# print (np.logspace(1, 7, num=ki))

+------------------+-------+------------------+
|       name       | index |      value       |
+------------------+-------+------------------+
|   (intercept)    |  None |  18993.4272128   |
|     bedrooms     |  None |  7936.96767903   |
| bedrooms_square  |  None |  936.993368193   |
|    bathrooms     |  None |  25409.5889341   |
|   sqft_living    |  None |  39.1151363797   |
| sqft_living_sqrt |  None |  1124.65021281   |
|     sqft_lot     |  None | 0.00348361822299 |
|  sqft_lot_sqrt   |  None |  148.258391011   |
|      floors      |  None |   21204.335467   |
|  floors_square   |  None |  12915.5243361   |
|    waterfront    |  None |  601905.594545   |
|       view       |  None |  93312.8573119   |
|    condition     |  None |  6609.03571245   |
|      grade       |  None |  6206.93999188   |
|    sqft_above    |  None |  43.2870534193   |
|  sqft_basement   |  None |  122.367827534   |
|     yr_built     |  None |  9.43363539372   |
|   yr_renovated   |  None |  56.0720034

In [120]:
1e10

10000000000.0

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [128]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [119]:
l1_penalty_values = np.logspace(8, 10, num=20)
print(l1_penalty_values)

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [131]:
ki = len(l1_penalty_values)
penalties = np.empty(ki)
nonzeros = np.empty(ki)
results = np.full(ki,np.finfo(np.float).max)
models = list()
i = 0;
l1_penalty_max = np.finfo(np.float).max
l1_penalty_min = 0
for l1_penalty in l1_penalty_values:
    m = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l1_penalty=l1_penalty, l2_penalty=0., 
                                              verbose = False)
    models.append(m)
    predict = m.predict(validation)
    score = predict - validation['price'] 
    validation_rss = (score*score).sum()
    
    nonzeros[i] = m['coefficients']['value'].nnz()
        
    results[i] = validation_rss
    penalties[i] = l1_penalty
    
    if (nonzeros[i] > max_nonzeros):
       l1_penalty_min = max(l1_penalty_min, l1_penalty)
    if (nonzeros[i] < max_nonzeros):
       l1_penalty_max = min(l1_penalty_max, l1_penalty)


    
    if (np.amin(results) == validation_rss):
        min_l1 = l1_penalty
        min_result = validation_rss
        min_i = i
    print (i, l1_penalty, validation_rss, nonzeros[i], l1_penalty_min, l1_penalty_max)
    i += 1

(0, 100000000.0, 627492659875075.5, 18.0, 100000000.0, 1.7976931348623157e+308)
(1, 127427498.57031322, 628210516770747.8, 18.0, 127427498.57031322, 1.7976931348623157e+308)
(2, 162377673.91887242, 629176689541061.1, 18.0, 162377673.91887242, 1.7976931348623157e+308)
(3, 206913808.11147901, 630650082718673.8, 18.0, 206913808.11147901, 1.7976931348623157e+308)
(4, 263665089.87303555, 632940229286984.0, 17.0, 263665089.87303555, 1.7976931348623157e+308)
(5, 335981828.62837881, 636268140229529.0, 17.0, 335981828.62837881, 1.7976931348623157e+308)
(6, 428133239.8719396, 641261198311188.2, 17.0, 428133239.8719396, 1.7976931348623157e+308)
(7, 545559478.11685145, 648983455376338.2, 17.0, 545559478.11685145, 1.7976931348623157e+308)
(8, 695192796.17755914, 660962217696068.4, 17.0, 695192796.17755914, 1.7976931348623157e+308)
(9, 885866790.41008317, 677261520727858.6, 16.0, 885866790.41008317, 1.7976931348623157e+308)
(10, 1128837891.6846883, 701046815867077.6, 15.0, 1128837891.6846883, 1.7976

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [132]:
# min(12,2)

# l1_penalty_min = largest l1_penalty
#   where more non-zeros than 7
#   if (non-zeros > 7):
#       l1_penalty_min = max(l1_penalty_min, l1_penalty)

# l1_penalty_max = smallest l1_penalty
#   where fewer non-zeros than 7
#   if (non-zeros < 7):
#       l1_penalty_max = min(l1_penalty_min, l1_penalty)


print (l1_penalty_min, l1_penalty_max)
# l1_penalty_min = 
# l1_penalty_max = 



(2976351441.6313133, 3792690190.7322536)


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [ ]:
# max = 3,792,690,190 = 3,792,000,000 = 3792000000
# min = 2,976,351,441 = 2,976,000,000 = 2976000000

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [138]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [144]:
ki = len(l1_penalty_values)
penalties = np.empty(ki)
nonzeros = np.empty(ki)
results = np.full(ki,np.finfo(np.float).max)
models = list()
i = 0;
for l1_penalty in l1_penalty_values:
    m = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l1_penalty=l1_penalty, l2_penalty=0., 
                                              verbose = False)
    models.append(m)
    
    nonzeros[i] = m['coefficients']['value'].nnz()
    
    if (nonzeros[i] == max_nonzeros):
        predict = m.predict(validation)
        score = predict - validation['price'] 
        validation_rss = (score*score).sum()
        results[i] = validation_rss
        penalties[i] = l1_penalty

        if (np.amin(results) == validation_rss):
            min_l1 = l1_penalty
            min_result = validation_rss
            min_i = i
            
    print (i, l1_penalty, validation_rss, nonzeros[i], min_l1, min_i)
    i += 1

(0, 2976351441.6313133, 1060799531763287.8, 10.0, 100000000.0, 0)
(1, 3019316638.9524155, 1060799531763287.8, 10.0, 100000000.0, 0)
(2, 3062281836.2735176, 1060799531763287.8, 10.0, 100000000.0, 0)
(3, 3105247033.5946198, 1060799531763287.8, 10.0, 100000000.0, 0)
(4, 3148212230.9157219, 1060799531763287.8, 10.0, 100000000.0, 0)
(5, 3191177428.236824, 1060799531763287.8, 10.0, 100000000.0, 0)
(6, 3234142625.5579262, 1060799531763287.8, 10.0, 100000000.0, 0)
(7, 3277107822.8790283, 1060799531763287.8, 10.0, 100000000.0, 0)
(8, 3320073020.2001305, 1060799531763287.8, 8.0, 100000000.0, 0)
(9, 3363038217.5212326, 1060799531763287.8, 8.0, 100000000.0, 0)
(10, 3406003414.8423347, 1060799531763287.8, 8.0, 100000000.0, 0)
(11, 3448968612.1634369, 1046937488751711.1, 7.0, 3448968612.1634369, 11)
(12, 3491933809.484539, 1051147625612860.9, 7.0, 3448968612.1634369, 11)
(13, 3534899006.8056412, 1055992735342999.1, 7.0, 3448968612.1634369, 11)
(14, 3577864204.1267428, 1060799531763287.8, 7.0, 344896

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [145]:
min_l1
# 3448968612 = 3448,968,612 = 3448,000,000 = 3448000000

3448968612.1634369

In [146]:
models[min_i].coefficients.print_rows(100)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x